# Csci 5523 Final Project
## Rona Response Team

In [ ]:
import numpy as np
import pandas as pd
import squarify
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree

In [ ]:
raw_data = pd.read_csv('all-states-history.csv')
raw_data.head()
raw_data.columns

In [ ]:
columns_to_show = ['date','state','death','deathConfirmed', 'deathIncrease','hospitalizedCurrently','hospitalizedIncrease','negative','positive','positiveIncrease','totalTestResults','totalTestResultsIncrease']
data = raw_data[columns_to_show]
data.head()

In [ ]:
MN_data = data[data['state']=='MN']
MN_data.head()

In [ ]:
MN_data.to_csv('MN_Data.csv')

In [ ]:
MN_data = pd.read_csv('MN_data.csv')
cols = MN_data.columns
MN_data = MN_data[cols[1:]]
MN_data.head()

In [ ]:
MN_fall = MN_data[0:90]
MN_summer = MN_data[90:182]
MN_spring = MN_data[182:]

In [ ]:
def discretize_data(arr, k):
    out = pd.DataFrame({'date': arr['date']})
    out['state'] = arr['state']
    cols = arr.columns[2:]
    for i in cols:
        bins = pd.cut(arr[i], k, 'retbins' == True, labels = list(range(k)))
        bin_range = pd.cut(arr[i],k)
        for j in range(k):
            count = 0
            out[i + ' bin'+str(bin_range.loc[count])] = np.zeros(len(arr))
            for row in arr.index:
                if bins.loc[row] == j:
                    out.loc[row, i + ' bin'+ str(bin_range.loc[count])] = 1
                count += 1
    out = out.fillna(0)
    return out      

In [ ]:
new = discretize_data(MN_fall,4)

In [ ]:
new.head()

In [ ]:
new.describe()